# **Reti Neurali**

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello.

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati

print("Shape x_train:", x_train.shape)
print("Shape y_train:", y_train.shape)
print("Shape x_test:", x_test.shape)
print("Shape y_test:", y_test.shape)



In [ ]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati

# svolgimento...

x_train_flat = x_train.reshape((x_train.shape[0], -1))  # (50000, 3072)
x_test_flat = x_test.reshape((x_test.shape[0], -1))     # (10000, 3072)

# Inizializzazione dello StandardScaler
scaler = StandardScaler()

# Adattamento dello scaler ai dati di addestramento
x_train_scaled = scaler.fit_transform(x_train_flat)

# Applicazione della trasformazione ai dati di test
x_test_scaled = scaler.transform(x_test_flat)

# Visualizzazione delle nuove shape
print("Shape x_train_scaled:", x_train_scaled.shape)
print("Shape x_test_scaled:", x_test_scaled.shape)

## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio:

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza.

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP

# svolgimento...
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=20, random_state=42, verbose=True)

# Allenare MLP

# svolgimento...
mlp.fit(x_train_scaled, y_train)





# Valutare MLP

# svolgimento...

y_pred = mlp.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)



# Stampare l' accuratezza

# svolgimento...
print("Accuratezza sul test set:", accuracy)

## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche

# svolgimento...

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creazione di un MLP più complesso
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),  # Tre layer nascosti con neuroni decrescenti
    activation='relu',                  # Funzione di attivazione
    solver='adam',                      # Ottimizzatore
    learning_rate_init=0.001,           # Learning rate iniziale
    max_iter=50,                        # Numero massimo di iterazioni
    random_state=42,
    verbose=True
)

# Addestramento del modello
mlp.fit(x_train_scaled, y_train)

# Predizione sui dati di test
y_pred = mlp.predict(x_test_scaled)

# Calcolo dell'accuratezza
accuracy = accuracy_score(y_test, y_pred)

# Stampa dell'accuratezza
print("Accuratezza sul test set:", accuracy)


## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting.

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


n_total_epochs = 100
patience = 10
tolerance = 1e-4

best_test_accuracy = 0.0
epochs_without_improvement = 0


#svolgimento:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

#nota: È una buona pratica usare un validation set separato per applicare l’early stopping, così da evitare l’overfitting ai dati di test.
# Splitting del training set in training + validation
X_train, X_val, y_train_split, y_val = train_test_split(
    x_train_scaled, y_train, test_size=0.2, random_state=42
)

# Parametri early stopping
n_total_epochs = 100
patience = 10
tolerance = 1e-4

# Inizializzazione
best_val_accuracy = 0.0
epochs_without_improvement = 0
train_accuracies = []
val_accuracies = []

# Modello MLP con warm start
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=1,
    warm_start=True,
    random_state=42,
    verbose=False
)

# Ciclo di addestramento
for epoch in range(n_total_epochs):
    mlp.fit(X_train, y_train_split)

    # Valutazione su train e validation
    train_acc = accuracy_score(y_train_split, mlp.predict(X_train))
    val_acc = accuracy_score(y_val, mlp.predict(X_val))

    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1} - Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

    # Early stopping
    if val_acc > best_val_accuracy + tolerance:
        best_val_accuracy = val_acc
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"\nEarly stopping a epoch {epoch+1}")
        break

# Valutazione finale su test set
y_test_pred = mlp.predict(x_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\nAccuratezza finale sul test set: {test_accuracy:.4f}")


